In [2]:
from transformers import AutoModelForSequenceClassification,AutoModelForCausalLM, AutoTokenizer,TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# 加载 Llama2-7B
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"  # 适用于 GPT 类模型
)
model = get_peft_model(model, lora_config)  # 应用 LoRA
tokenizer = AutoTokenizer.from_pretrained(model_name)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 配置 LoRA 适配层
# 应用 LoRA
model = get_peft_model(model, lora_config)

/Users/haoyuhuang/opt/miniconda3/envs/ata/lib/python3.8/site-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'google-bert/bert-base-chinese' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [4]:
device = model.device

In [5]:
from datasets import load_dataset
import torch
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").view(-1).float().to(device)  # ✅ 1D float
        outputs = model(**inputs)
        logits = outputs.logits.view(-1)  # ✅ 1D logits

        # 计算 BCEWithLogitsLoss
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [6]:



# 3️⃣ 处理数据
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = [float(label) for label in examples["label"]]  # 确保 label 是 float
    return inputs

dataset = load_dataset("json", data_files={"train": "train.json", "test": "train.json"})
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 4️⃣ 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    learning_rate=2e-5
)



# 6️⃣ 训练
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)
trainer.train()


Map: 100%|██████████| 6/6 [00:00<00:00, 223.85 examples/s]
                                             
 20%|██        | 1/5 [00:07<00:23,  5.95s/it]Checkpoint destination directory ./results/checkpoint-1 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.7300662994384766, 'eval_runtime': 1.5266, 'eval_samples_per_second': 3.93, 'eval_steps_per_second': 0.655, 'epoch': 1.0}


                                             
 40%|████      | 2/5 [00:14<00:19,  6.55s/it]Checkpoint destination directory ./results/checkpoint-2 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.7275132536888123, 'eval_runtime': 1.6185, 'eval_samples_per_second': 3.707, 'eval_steps_per_second': 0.618, 'epoch': 2.0}


                                             
 60%|██████    | 3/5 [00:21<00:13,  6.70s/it]Checkpoint destination directory ./results/checkpoint-3 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

                                             
 80%|████████  | 4/5 [00:28<00:06,  6.76s/it]Checkpoint destination directory ./results/checkpoint-4 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.7244434356689453, 'eval_runtime': 1.5554, 'eval_samples_per_second': 3.858, 'eval_steps_per_second': 0.643, 'epoch': 4.0}


                                             
100%|██████████| 5/5 [00:35<00:00,  7.03s/it]

{'eval_loss': 0.7238247394561768, 'eval_runtime': 1.5935, 'eval_samples_per_second': 3.765, 'eval_steps_per_second': 0.628, 'epoch': 5.0}
{'train_runtime': 35.1354, 'train_samples_per_second': 0.854, 'train_steps_per_second': 0.142, 'train_loss': 0.7388360500335693, 'epoch': 5.0}


TrainOutput(global_step=5, training_loss=0.7388360500335693, metrics={'train_runtime': 35.1354, 'train_samples_per_second': 0.854, 'train_steps_per_second': 0.142, 'train_loss': 0.7388360500335693, 'epoch': 5.0})

In [8]:
import torch
import torch.nn.functional as F

def predict(text):
    # 确保模型在正确的设备上
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 预处理输入文本
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)
    
    # 计算 logits（模型原始输出）
    with torch.no_grad():
        logits = model(**inputs).logits  # logits 形状: (1, 1)
        prob = torch.sigmoid(logits)  # 计算概率（0~1）

    return prob.item()  # 返回单个数值




In [21]:
# **测试预测**
text1 = "用户在过去三个月内多次逾期还款，最近一次支付金额低于最低还款额"
text2 = "用户信用评分良好，稳定工作，房贷已结清"

print(f"Text1 逾期概率: {predict(text1):.4f}")  
print(f"Text2 逾期概率: {predict(text2):.4f}")  

Text1 逾期概率: 0.4461
Text2 逾期概率: 0.4351
